In [1]:
import glob
import numpy as np
import cartesian
import lsq_transform
import pivot_calibration

In [2]:
runtype = 0
run = 5
if True:
    letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']
    type = ['debug', 'unknown']
    data_dir = "DATA/"

    # File IO
    empivot = glob.glob(data_dir + 'pa1-' + type[runtype] + '-' + letters[run] + '-empivot.txt')
    empivot_file = open(empivot[0], "r")
    empivot_lines = empivot_file.read().splitlines()
    empivot_data = []
    for num in range(len(empivot_lines)):
        empivot_data.append( empivot_lines[num].split(',') )
        for i in range(len(empivot_data[-1])):
            empivot_data[-1][i] = empivot_data[-1][i].strip()

    Ng = int(empivot_data[0][0])
    Nframes = int(empivot_data[0][1])
    empivot_data = np.asarray(empivot_data[1:]).astype(float)
    
    # Use the first “frame” of pivot calibration data to define a local “probe” coordinate system
    G = empivot_data[ : Ng , : ]
    G0 = np.sum(G, axis=0)/Ng
    g = G - G0
    
    G_stack = np.zeros(1)
    p_stack = np.zeros(1)

In [3]:
print("Nd : ",Ng)
print("Nd : ",Nframes)

Nd :  6
Nd :  12


In [4]:
Rotation = np.zeros((3,3))
Translation = np.zeros((3,1))
Transform = []
for i in range(Nframes):
        G = empivot_data[ i*Ng : (i+1)*Ng , : ]
        F = lsq_transform.get_lsq_transform( g , G )
        Transform.append(F)
        
        rotation = F[0:3,0:3]
        #print( rotation.shape )
        Rotation = np.vstack( ( Rotation , rotation ) )
        #Translation .append()
        # need to save all those data and applied hw3 Q1C
        translation = F[ 0:3 , 3 ]
        Translation = np.vstack( ( Translation , translation.reshape(3,1) ) )


In [5]:
Transform = np.asarray( Transform ).astype(float)

In [6]:
Rotation = Rotation[3:]

In [7]:
Rotation.shape

(36, 3)

In [8]:
Translation = Translation[3:]

In [9]:
Translation.shape

(36, 1)

In [10]:
n = (Rotation.shape)[0] // 3
right_half = -1 * np.eye(3)
for i in range(n-1):
    right_half = np.vstack( ( right_half , -1 * np.eye(3) ) )

In [11]:
right_half.shape

(36, 3)

In [12]:
A = np.concatenate((Rotation,right_half),1)

In [13]:
A.shape

(36, 6)

In [14]:
pivot_calibration.pivot_calibration( Rotation , Translation )

array([[ -32.11230755],
       [  54.51702493],
       [  79.88695912],
       [-209.7933234 ],
       [-201.55334027],
       [-199.53958457]])